In [2]:
# Local Import
import numpy as np
import pandas as pd
import pickle

In [80]:
%cd /home/nbuser/library/DP/Alaska/
p_df1 = pd.read_csv('Alaska2000_5PUMS_Household.csv')
p_df2 = pd.read_csv('Alaska2000_5PUMS_Person.csv')
df = pd.merge(p_df2, p_df1, on='serial_no')
df = df.fillna({'RENT':0, 'VALUE':0, 'INCTOT':0})
df = df.rename(columns={'SUBSAMPL': 'Cell'})

/home/nbuser/library/DP/Alaska


In [ ]:
np.nanpercentile(df.INCTOT, 99.5)

In [81]:
rent = [df.RENT[i] / 4.35 * 1000 for i in range(len(df.RENT))]
renta = np.asarray(rent)
renta[0]=0
renta[np.logical_and(np.greater_equal(renta, 1), np.less(renta, 10000))]=1
renta[np.logical_and(np.greater_equal(renta, 10000), np.less(renta, 14999))]=2
renta[np.logical_and(np.greater_equal(renta, 15000), np.less(renta, 19999))]=3
renta[np.logical_and(np.greater_equal(renta, 20000), np.less(renta, 24999))]=4
renta[np.logical_and(np.greater_equal(renta, 25000), np.less(renta, 29999))]=5
renta[np.logical_and(np.greater_equal(renta, 30000), np.less(renta, 34999))]=6
renta[np.logical_and(np.greater_equal(renta, 35000), np.less(renta, 39999))]=7
renta[np.logical_and(np.greater_equal(renta, 40000), np.less(renta, 49999))]=8
renta[np.logical_and(np.greater_equal(renta, 50000), np.less(renta, 59999))]=9
renta[np.logical_and(np.greater_equal(renta, 60000), np.less(renta, 69999))]=10
renta[np.logical_and(np.greater_equal(renta, 70000), np.less(renta, 79999))]=11
renta[np.logical_and(np.greater_equal(renta, 80000), np.less(renta, 89999))]=12
renta[np.logical_and(np.greater_equal(renta, 90000), np.less(renta, 99999))]=13
renta[np.logical_and(np.greater_equal(renta, 100000), np.less(renta, 124999))]=14
renta[np.logical_and(np.greater_equal(renta, 125000), np.less(renta, 149999))]=15
renta[np.logical_and(np.greater_equal(renta, 150000), np.less(renta, 174999))]=16
renta[np.logical_and(np.greater_equal(renta, 175000), np.less(renta, 199999))]=17
renta[np.logical_and(np.greater_equal(renta, 200000), np.less(renta, 249999))]=18
renta[np.logical_and(np.greater_equal(renta, 250000), np.less(renta, 299999))]=19
renta[np.logical_and(np.greater_equal(renta, 300000), np.less(renta, 399999))]=20
renta[np.logical_and(np.greater_equal(renta, 400000), np.less(renta, 499999))]=21
renta[np.logical_and(np.greater_equal(renta, 500000), np.less(renta, 749999))]=22
renta[np.logical_and(np.greater_equal(renta, 750000), np.less(renta, 999999))]=23
renta[np.greater_equal(renta, 100000)]=24

In [82]:
df.RENT = renta
df.VALUE = df.RENT + df.VALUE
df.RENT[df.RENT > 0] = True
df.RENT[df.RENT == 0] = False
df = df[df.TENURE < 4]
df = df.dropna(subset=['YRBUILT'])
# df = df[df.INCTOT >= np.nanpercentile(df.INCTOT, 90)]
df = df[df.INCTOT >= np.nanpercentile(df.INCTOT, 99.5)]

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [83]:
DF = df.drop(['SEX', 'RACE3', 'RENT', 'TENURE'], axis=1)
# df2 = df1.drop_duplicates('serial_no')

#Regression 1 Income Total v AGE
IA = DF.drop(['YRBUILT', 'HINC', 'VALUE', 'serial_no'], axis=1)
IA = IA.reindex(columns=['Cell', 'INCTOT', 'age', 'PUMA5'])

# # Regression 2 Income Total v VALUE
# IV = DF.drop(['HINC', 'YRBUILT', 'age', 'serial_no'], axis=1)
# IV = IV.reindex(columns=['Cell', 'INCTOT', 'VALUE', 'PUMA5'])

# Regression 3 Income Total v AGE v VALUE
IAV = DF.drop(['YRBUILT', 'HINC', 'serial_no'], axis=1)
IAV = IAV.reindex(columns=['Cell', 'INCTOT', 'age', 'VALUE', 'PUMA5'])

In [114]:
# x = UI(X, N)
y = UI(Y, N)
x = UII(X)

In [39]:
# Define Methods
# Regression
rO = ["OLS", "SE"]
rW = ["OLS", "Winsorize", "SE"]
rM = ["MM", "Winsorize", "SE"]
rMM = ["MM", "Winsorize", "SE", "500"]
rS = ["SMDM", "Winsorize", "SE"]

In [26]:
# Import Algorithms (& Libraries)
%cd /home/nbuser/library/DP/
%run Methods.ipynb

/home/nbuser/library/DP


In [6]:
R0, O0 = Methods(X, Y, N, v, rM)()

In [55]:
print(npl(R0.T[0].T)[1])
npl(O0.T[0].T)[1]

[ 0.28452297  0.00825714 -0.19411851]


array([[ 0.13065006,  0.10989919,  0.16490551],
       [ 0.26791503,  0.02984753, -0.2078137 ],
       [ 0.24341557, -0.04506442,  0.20647748],
       [ 0.31728789, -0.03278297, -0.21488934],
       [ 0.43685626, -0.20360153, -0.19432445],
       [ 0.22167454, -0.05925551,  0.41516152],
       [ 0.24147231,  0.00211657,  0.04241238],
       [ 0.18189139, -0.01557427,  0.45366545]])

In [61]:
print(npl(R1.T[0].T)[1])
npl(O1.T[0].T)[1]

[ 0.28452297  0.00825714 -0.19411851]


array([[ 0.13065006,  0.10989919,  0.16490551],
       [ 0.29239007, -0.0008149 , -0.24023555],
       [ 0.25626936, -0.07803134,  0.22722366],
       [ 0.32673185, -0.05084648, -0.21920927],
       [ 0.45655529, -0.23371442, -0.17105809],
       [ 0.217045  , -0.06143128,  0.43174843],
       [ 0.24145392,  0.00213798,  0.04243902],
       [ 0.18570591, -0.02982647,  0.47471907]])

In [71]:
ATT = npl(R1.T[0].T)[1]
ATAT = npl(O1.T[0].T)[1][1]
la.norm(ATT - ATAT)

0.047654740342158136

In [2]:
# Define Methods
LBD = ["Laplace", "BETA", "DWORK"]
LBM = ["Laplace", "BETA", "MOSE"]
NBD = ["Gaussian", "BETA", "DWORK"]
NBM = ["Gaussian", "BETA", "MOSE"]

LS = ["Laplace", "SE"]
NS = ["Gaussian", "SE"]

In [3]:
%cd /home/nbuser/library/DP/Data/
with open("99IAVC.pickle", "rb") as handle:
    X, Y, N, v = pickle.load(handle)
with open("99IAVT.pickle", "rb") as handle:
    tr, to = pickle.load(handle)

/home/nbuser/library/DP/Data


In [4]:
with open("99IAVS.pickle", "rb") as handle:
    r, o = pickle.load(handle)

In [7]:
# Test Dwork
dwr = DWORK(tr, r, o, 0.5, N, LBD)

In [177]:
ld = MSE(tr, r, o, N, LBD)()
lm = MSE(tr, r, o, N, LBM)()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:26: RuntimeWarning: All-NaN axis encountered
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:32: RuntimeWarning: Mean of empty slice


In [179]:
nd = MSE(tr, r, o, N, NBD)()
nm = MSE(tr, r, o, N, NBM)()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:26: RuntimeWarning: All-NaN axis encountered
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:32: RuntimeWarning: Mean of empty slice


In [181]:
with open('MSE_IAVS.pickle', 'wb') as handle:
    pickle.dump((ld, lm, nd, nm), handle, pickle.HIGHEST_PROTOCOL)